In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata

In [2]:
tqdm.pandas()

path_interim = "../data/interim/pjud"
path_processed = "../data/processed/pjud"

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# DATA de TRIBUNALES OBTENIDAS DEL COT

In [3]:
# Analizo contra los datos extraidos en COT
df_tribunales = pd.read_feather(f"{path_processed}/ListadoTribunales_feather")
tribunales = df_tribunales['TRIBUNAL'].unique()

# DATA INGRESO Y TERMINO ROL PENAL

### INGRESOS X ROL

In [4]:
df_ingresos_rol = pd.read_feather(f"{path_interim}/IngresosRol_feather")
df_ingresos_rol.head()

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO
0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,1-2018,Ordinaria,2018-01-02
1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,2-2018,Ordinaria,2018-01-03
2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,3-2018,Ordinaria,2018-01-03
3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,4-2018,Ordinaria,2018-01-03
4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,5-2018,Ordinaria,2018-01-03


In [5]:
# Normalizo nombres

df_ingresos_rol['TRIBUNAL'] = df_ingresos_rol['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 2980977/2980977 [00:17<00:00, 167880.08it/s]


In [6]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_ingresos_rol['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

### TERMINO X ROL

In [7]:
df_termino_rol = pd.read_feather(f"{path_interim}/TerminoRol_feather")
df_termino_rol.head()

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO
0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,2018-04-02,2018-06-01,60,SENTENCIA
1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2018,Ordinaria,2018-04-02,2018-08-29,149,SENTENCIA
2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,10-2018,Ordinaria,2018-01-10,2018-03-14,63,SENTENCIA
3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,102-2018,Ordinaria,2018-04-02,2018-05-29,57,SENTENCIA
4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,103-2018,Ordinaria,2018-04-02,2018-05-30,58,SENTENCIA


In [8]:
# Normalizo nombres

df_termino_rol['TRIBUNAL'] = df_termino_rol['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 3357121/3357121 [00:19<00:00, 171141.50it/s]


In [9]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_termino_rol['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

### CONSOLIDACION DE DATA

In [10]:
pd.set_option('display.max_columns', 100)

In [11]:
df_consolidado = pd.merge(df_ingresos_rol, df_termino_rol, how='outer', on=['COD. TRIBUNAL','RIT'])

In [12]:
df_consolidado

,index_x,COD. CORTE_x,CORTE_x,COD. TRIBUNAL,TRIBUNAL_x,RIT,TIPO CAUSA_x,FECHA INGRESO_x,index_y,COD. CORTE_y,CORTE_y,TRIBUNAL_y,TIPO CAUSA_y,FECHA INGRESO_y,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO
0,0.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,1-2018,Ordinaria,2018-01-02,21.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,2018-01-02,2018-03-05,62.0,SENTENCIA
1,1.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,2-2018,Ordinaria,2018-01-03,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
2,2.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,3-2018,Ordinaria,2018-01-03,223.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,2018-01-03,2018-02-20,48.0,SENTENCIA
3,3.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,4-2018,Ordinaria,2018-01-03,335.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,2018-01-03,2018-03-26,82.0,SENTENCIA
4,4.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,5-2018,Ordinaria,2018-01-03,426.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,2018-01-03,2018-04-18,105.0,SENTENCIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4306303,NaN,NaN,NaN,1355,NaN,85-2014,NaN,NaT,679951.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,2014-12-01,2015-01-24,54.0,SENTENCIA
4306304,NaN,NaN,NaN,1355,NaN,86-2014,NaN,NaT,679953.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,2014-12-02,2015-01-31,60.0,SENTENCIA
4306305,NaN,NaN,NaN,1355,NaN,87-2014,NaN,NaT,679954.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,2014-12-09,2015-01-28,50.0,SENTENCIA
4306306,NaN,NaN,NaN,1355,NaN,88-2014,NaN,NaT,679955.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,2014-12-30,2015-02-02,34.0,SENTENCIA


## CARGA DATA A ARCHIVOS FUTHER

In [13]:
# Directorio donde se guardaran archivos feather

os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_ingresos_rol.to_feather(f'{path_processed}/IngresosRol_feather')
df_termino_rol.to_feather(f'{path_processed}/TerminoRol_feather')
df_consolidado.to_feather(f'{path_processed}/ConsolidadoRol_feather')
